In [1]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd4843 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 24 × 12th Gen Intel(R) Core(TM) i9-12900K
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, alderlake)
Threads: 4 default, 0 interactive, 2 GC (on 24 virtual cores)
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
using Pkg; Pkg.status()

Status `C:\Users\o6m1g\.julia\environments\v1.10\Project.toml`
  [99985d1d] AbstractGPs v0.5.21
⌃ [c75e803d] AdaptiveRejectionSampling v0.1.2
⌅ [0bf59076] AdvancedHMC v0.5.5
⌅ [5b7e9947] AdvancedMH v0.7.5
  [488c2830] BSplines v0.3.3
  [0a1fb500] BlockDiagonals v0.1.42
  [c88b6f0a] BridgeStan v2.5.0
⌃ [336ed68f] CSV v0.10.14
  [aaaa29a8] Clustering v0.15.7
  [8f4d0f93] Conda v1.10.2
⌃ [a93c6f00] DataFrames v1.6.1
  [055956cb] DiffEqPhysics v3.15.0
⌃ [0c46a032] DifferentialEquations v7.10.0
⌃ [31c24e10] Distributions v0.25.109
  [cc61a311] FLoops v0.2.2
⌃ [587475ba] Flux v0.14.16
  [38e38edf] GLM v1.9.0
  [bd48cda9] GraphRecipes v0.5.13
⌃ [34004b35] HypergeometricFunctions v0.3.23
⌃ [09f84164] HypothesisTests v0.11.0
⌃ [7073ff75] IJulia v1.25.0
  [4138dd39] JLD v0.13.5
  [682c06a0] JSON v0.21.4
⌃ [b964fa9f] LaTeXStrings v1.3.1
⌃ [6fdf6af0] LogDensityProblems v2.1.1
  [10e44e05] MATLAB v0.8.4
⌃ [c7f686f2] MCMCChains v5.7.1
⌃ [86f7a689] NamedArrays v0.10.2
⌃ [bac558e1] OrderedCollections 

# Examples

In [7]:
include("Init.jl")

## Yield curve

In [8]:
df = CSV.read("data/yield.csv", DataFrame);

In [9]:
mutable struct Data_DNS
    y::Matrix{Float64}
    T::Int64
    K::Int64
    τ_axis::Vector{Int64}
    X::Matrix{Float64}
    dates::Vector
    ρ::Float64
end

In [36]:
function generate_data(T::Int=size(df,1))
    _df = df[1:T,2:end]
    τ_axis = [parse(Int64, τ_string) for τ_string in names(df[:,2:end])]
    τ_axis = 12τ_axis # in months

    x(τ) = (λ_t = 0.0609;
        _m = τ * λ_t;
        _exp = exp(-_m);
        _s = (1 - _exp) / _m;
        [1, _s, _s - _exp]
    )
    
    global data = Data_DNS(
        _df |> Matrix,
        size(_df, 1),
        size(_df, 2),
        τ_axis,
        vecvec2mat(x.(τ_axis)),
        df.index,
        1.
    )
end

generate_data (generic function with 2 methods)

In [37]:
generate_data()

Data_DNS([0.293 0.905 … 2.49 2.643; 0.286 0.944 … 2.537 2.666; … ; 0.048 0.233 … 1.404 1.662; 0.094 0.312 … 1.516 1.79], 293, 5, [24, 60, 120, 240, 360], [1.0 0.52554392871227 0.2936789349181237; 1.0 0.2665880208221924 0.24070064890648354; … ; 1.0 0.0684181411392243 0.06841769203012686; 1.0 0.0456121145639038 0.04561211426293063], [Date("1999-09-01"), Date("1999-10-01"), Date("1999-11-01"), Date("1999-12-01"), Date("2000-01-01"), Date("2000-02-01"), Date("2000-03-01"), Date("2000-04-01"), Date("2000-05-01"), Date("2000-06-01")  …  Date("2023-04-01"), Date("2023-05-01"), Date("2023-06-01"), Date("2023-07-01"), Date("2023-08-01"), Date("2023-09-01"), Date("2023-10-01"), Date("2023-11-01"), Date("2023-12-01"), Date("2024-01-01")], 1.0)

In [38]:
include("LEO.jl")

### Initial draw

In [39]:
Θ = Params(); chain = Params[]
@showprogress for r in 1:12_000
    Θ |> GibbsScan
    if (r > 2_000) & (r % 10 == 0)
        push!(chain, Θ |> deepcopy)
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:03:36


In [40]:
save("output/DNS/chain.jld", "data", chain)

### Naive LEO

In [287]:
leave_ends = size(df, 1):-1:size(df, 1)-20

293:-1:273

In [283]:
times = Float64[]

@showprogress for T in leave_ends
    """Create data"""
    generate_data(T-1) # exclude last
    
    """MCMC"""
    Θ = Params(); chain = Params[]
    time = @elapsed for r in 1:12_000
        Θ |> GibbsScan
        if (r > 2_000) & (r % 10 == 0)
            push!(chain, Θ |> deepcopy)
        end
    end
    push!(times, time)

    save("output/DNS/chain-LEO-$(T).jld", "data", chain)
    save("output/DNS/times-LEO.jld", "data", times)
    # break
end

Progress: 100%|█████████████████████████████████████████| Time: 1:13:20


### SMC-LEO

In [324]:
leave_ends

293:-1:273

In [325]:
generate_data();

In [326]:
let
    Random.seed!(1)
    chain = load("output/DNS/chain.jld")["data"]

    # Define problem dimensions
    R = length(chain)
    L = 5

    # Obtain unconstrained prior draw
    Θ_0 = chain
    
    # Initialize containers
    particles = NamedArray(
        repeat([Θ_0], length(leave_ends), L+1),
        (leave_ends, 0:L),
        (:T, :l))
    ϕ_history = NamedArray(
        zeros(length(leave_ends), L+1),
        (leave_ends, 0:L),
        (:T, :l))
    L_history = NamedArray(repeat([-1], length(leave_ends)),
        leave_ends, :T)
    times = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    
    # Log weights normalizer
    function _normalize(_log_w::Vector{Float64})::Vector{Float64}
        # normalize log weights
        _w = exp.(_log_w .- maximum(_log_w))
        _w = _w / sum(_w)
        _w
    end

    # ESS computer
    function _ess(_w::Vector{Float64})::Float64
        1 / sum(@. exp(2 * log(_w)))
        # No: 1 / sum(@. exp(2 * _log_w))
    end
    
    # Log ratio
    function log_G(T::Int, l::Int, n::Int)::Float64
        _particle = particles[:T => T, :l => l-1][n]
        -logpdf(MvNormal(data.X * _particle.β[T,:], _particle.Σ_y), data.y[T,:])
    end

    for (T_i, T) in enumerate(leave_ends)
        @info T
        
        # Set initial values, starting index
        particles[:T => T, :l => 0] = T_i == 1 ? Θ_0 : particles[:T => leave_ends[T_i - 1], :l => L_history[:T => leave_ends[T_i - 1]]]
        ϕ_history[:T => T, :l => 0] = 0.
        break_flag = false
        data.ρ = 1
    
        log_weights_T = [log_G(T, 1, n) for n in 1:R]
        
        time = @elapsed @showprogress for l in 1:L
    
            ϕ_0 = ϕ_history[:T => T, :l => l-1]
            function _ϕ2reff(ϕ::Union{Float64, Int64})::Float64
                @assert 0 <= ϕ <= 1
                _log_weights = (ϕ - ϕ_0) * log_weights_T
                _log_weights = _log_weights .- maximum(_log_weights)
                _ESS = _log_weights |> _normalize |> _ess
                _ESS - 0.5R
            end
            # plot(0:0.01:1, _ϕ2reff) |> display
            
            # Initialize next distribution parameter
            ϕ_1 = nothing
            if _ϕ2reff(1) > 0 # Case 1: ESS is above threshold
                ϕ_1 = 1
                break_flag = true
            else # Case 2: find next distribution
                ϕ_1 = find_zero(_ϕ2reff, (ϕ_history[:T => T, :l => l-1] + 1e-4, 1), xtol=0.1, maxiters=10, verbose=false)
            end
            
            @info "Targeting ϕ=$(ϕ_1)"
            data.T = T
            data.ρ = 1 - ϕ_1
        
            # Compute log weights
            _log_weights = (ϕ_1 - ϕ_0) * log_weights_T
            _log_weights = _log_weights .- maximum(_log_weights)
            
            # MCMC kernel
            A_0 = wsample(1:R, _log_weights |> _normalize, R)
            _begin = particles[:T => T, :l => l-1][A_0] |> deepcopy
            _end = _begin |> deepcopy
            #@showprogress for n in 1:R
            p = ProgressMeter.Progress(R); @Threads.threads for n in 1:R
                _end[n] = _begin[n] |> GibbsScan_LEO^5
                ProgressMeter.next!(p)
                # @assert size(_end[n].β) == (T, 3)
            end
            
            particles[:T => T, :l => l] = _end
            ϕ_history[:T => T, :l => l] = ϕ_1
            if break_flag
                L_history[:T => T] = l
                break
            end
        end
        times[:T => T] = time
    end

    global particles = particles
    global ϕ_history = ϕ_history
    global L_history = L_history
    global times = times
end;

[ Info: 293
[ Info: Targeting ϕ=0.6313816406249998
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31                                                                                                                                                                 
[ Info: Targeting ϕ=1
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31
[ Info: 292
[ Info: Targeting ϕ=0.8760398437499999
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31
[ Info: Targeting ϕ=1
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31
[ Info: 291
[ Info: Targeting ϕ=0.35365371093749987
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31Progress:  98%|█████████████████████████████████████████|  ETA: 0:00:00                                                                                                                                                                    
[ Info: Targeting ϕ=0.7070718351364133
Progre

In [327]:
save("output/DNS/particles.jld", "data", particles |> Matrix{Vector{Params}}) # (T, L)
save("output/DNS/paths.jld", "data", ϕ_history |> Matrix{Float64}) # (T, L)
save("output/DNS/lengths.jld", "data", L_history |> Vector{Int64}) # T
save("output/DNS/times.jld", "data", times |> Vector{Float64}) # T

### PSIS-LEO

In [452]:
generate_data();

In [453]:
let
    Random.seed!(1)
    chain = load("output/DNS/chain.jld")["data"]

    # Define problem dimensions
    R = length(chain)

    # Obtain unconstrained prior draw
    Θ_0 = chain
    
    # Initialize containers
    particles = NamedArray(repeat([Θ_0], length(leave_ends)), leave_ends, :T)
    weights_psis = NamedArray(
        zeros(length(leave_ends), R),
        (leave_ends, 1:R),
        (:T, :n)
    )
    times = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    k̂s = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    
    # Log weights normalizer
    function _normalize(_log_w::Vector{Float64})::Vector{Float64}
        # normalize log weights
        _w = exp.(_log_w .- maximum(_log_w))
        _w = _w / sum(_w)
        _w
    end

    # ESS computer
    function _ess(_w::Vector{Float64})::Float64
        1 / sum(@. exp(2 * log(_w)))
        # No: 1 / sum(@. exp(2 * _log_w))
    end
    
    # Log ratio
    function log_G(T::Int, n::Int)::Float64
        _particle = Θ_0[n]
        -logpdf(MvNormal(data.X * _particle.β[T,:], _particle.Σ_y), data.y[T,:])
    end

    for (T_i, T) in enumerate(leave_ends)
        @info T
        
        # Compute log weights
        _log_weights = sum([log_G(t, n) for n in 1:R] for t in T:data.T)
        _log_weights = _log_weights .- maximum(_log_weights)
        
        # PSIS
        time = @elapsed _psis = psis(_log_weights; warn=false)
        weights_psis[:T => T] = _psis.log_weights |> _normalize
        k̂s[:T => T] = _psis.pareto_shape
        times[:T => T] = time
    end

    # global particles = particles
    global weights_psis = weights_psis
    global times_psis = times
    global k̂s_psis = k̂s
end;

[ Info: 293
[ Info: 292
[ Info: 291
[ Info: 290
[ Info: 289
[ Info: 288
[ Info: 287
[ Info: 286
[ Info: 285
[ Info: 284
[ Info: 283
[ Info: 282
[ Info: 281
[ Info: 280
[ Info: 279
[ Info: 278
[ Info: 277
[ Info: 276
[ Info: 275
[ Info: 274
[ Info: 273


## Figures

### LPD

In [454]:
generate_data()

Data_DNS([0.293 0.905 … 2.49 2.643; 0.286 0.944 … 2.537 2.666; … ; 0.048 0.233 … 1.404 1.662; 0.094 0.312 … 1.516 1.79], 293, 5, [24, 60, 120, 240, 360], [1.0 0.52554392871227 0.2936789349181237; 1.0 0.2665880208221924 0.24070064890648354; … ; 1.0 0.0684181411392243 0.06841769203012686; 1.0 0.0456121145639038 0.04561211426293063], [Date("1999-09-01"), Date("1999-10-01"), Date("1999-11-01"), Date("1999-12-01"), Date("2000-01-01"), Date("2000-02-01"), Date("2000-03-01"), Date("2000-04-01"), Date("2000-05-01"), Date("2000-06-01")  …  Date("2023-04-01"), Date("2023-05-01"), Date("2023-06-01"), Date("2023-07-01"), Date("2023-08-01"), Date("2023-09-01"), Date("2023-10-01"), Date("2023-11-01"), Date("2023-12-01"), Date("2024-01-01")], 1.0)

In [331]:
function Compute_LPD(c::Vector{Params}, T::Int64, weights::Vector{Float64})::Float64
    R = length(c)
    log_ℓ = 0.
    for n in 1:R
        β_T_sim = MvNormal(c[n].β[T-1,:], c[n].Σ_β) |> rand
        log_ℓ += weights[n] * logpdf(MvNormal(data.X * β_T_sim, c[n].Σ_y), data.y[T,:])
        
    end
    log_ℓ
end

Compute_LPD (generic function with 3 methods)

In [332]:
let
    Random.seed!(1)
    LPD = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    @showprogress for T in leave_ends
        chain_T = load("output/DNS/chain-LEO-$(T).jld")["data"]
        R = length(chain_T)
        LPD[:T => T] = Compute_LPD(chain_T, T, repeat([1/R], R))
    end
    global LPD_true = LPD
    # times = load("output/DNS/times-LEO.jld")["data"]
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


In [333]:
let
    Random.seed!(1)
    particles = load("output/DNS/particles.jld")["data"]
    L = size(particles, 2)-1
    particles = NamedArray(
        particles,
        (leave_ends, 0:L),
        (:T, :l))
    L_history = NamedArray(load("output/DNS/lengths.jld")["data"],
        leave_ends, :T)

    LPD = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    @showprogress for T in leave_ends
        chain_T = particles[:T => T, :l => L_history[:T => T]]
        R = length(chain_T)
        LPD[:T => T] = Compute_LPD(chain_T, T, repeat([1/R], R))
    end
    global LPD_smc = LPD
end;

In [334]:
let
    Random.seed!(1)
    chain = load("output/DNS/chain.jld")["data"]
    LPD = NamedArray(zeros(length(leave_ends)), leave_ends, :T)
    @showprogress for T in leave_ends
        LPD[:T => T] = Compute_LPD(chain, T, weights_psis[:T => T])
    end
    global LPD_psis = LPD
end;

In [527]:
let
    _data = [LPD_true LPD_smc LPD_psis]
    plot(leave_ends, cumsum(_data; dims=1),# ./ (1:size(_data,1)),
        label=["MCMC-LEO" "SMC-LEO" "PSIS-LEO"],
        color=[:blue :red :green],
        linestyle=[:solid :dot :dot],
        mswidth=0, markershape=[:pixel :o :square], markersize=3,
        linewidth=2, alpha=[1 0.8 0.7],
        legend=:topright
    )
    #_true_mean, _smc_mean, _psis_mean = _mean(_data; dims=1)
    #hline!([_true_mean], label="", color=:blue, linewidth=1)
    #hline!([_smc_mean], label="", color=:red, linewidth=1)
    #hline!([_psis_mean], label="", color=:green, linewidth=1)
    plt_1 = plot!()
    
    plot(leave_ends, cumsum(_data; dims=1) ./ (1:size(_data,1)),
        # label=["MCMC-LEO" "SMC-LEO" "PSIS-LEO"],
        color=[:blue :red :green],
        linestyle=[:solid :dot :dot],
        mswidth=0, markershape=[:pixel :o :square], markersize=3,
        linewidth=2, alpha=[1 0.8 0.7],
        legend=false
    )
    plt_2 = plot!()
   
    plot(plt_1, plt_2, layout=grid(2,1), size=(500,310),
        title=["Backward cumulative" "Backward running average"],
        xticks=(leave_ends,
            df.index[leave_ends] .|> (d -> month(d) == 1 ?  "$(month(d))\n" * L"\leftarrow" * "($(year(d)))     " : month(d))),
        #xflip=true,
    )
    savefig("img/yield-lpd-compare.pdf")
end

"C:\\Users\\o6m1g\\Documents\\GitHub\\SMC-LGO-CV-private\\img\\yield-lpd-compare.pdf"

In [499]:
let
    particles = load("output/DNS/particles.jld")["data"]
    L = size(particles, 2)-1
    ϕ_history = NamedArray(
        load("output/DNS/paths.jld")["data"],
        (leave_ends, 0:L),
        (:T, :l))
    L_history = NamedArray(load("output/DNS/lengths.jld")["data"],
        leave_ends, :T)

    _data = cumsum(LPD_smc; dims=1)
    plot(leave_ends, _data,
        label="SMC-LEO", color=:red, linestyle=:dot,
        mswidth=0, markershape=:o, linewidth=2, alpha=1,
        xticks=(leave_ends,
            df.index[leave_ends] .|> (d -> month(d) == 1 ?  "$(month(d))\n($(year(d)))" : month(d))),
    )

    _max, _min = maximum(_data), minimum(_data)

    for (T_i, T) in leave_ends |> enumerate
        _x = T .+ (1 .- ϕ_history[Name(T), Name.(1:L_history[:T => T]-1)])
        vline!(_x, label="", style=:dash, color=:black)
        for _x_i in _x
            _text = round(_x_i - T; digits=2)
            if T_i == 1
                _text = L"ρ" * "=$(_text)"
            end
            annotate!(_x_i+0.23, _max, text(_text, :black, 6, rotation=-90))
        end
    end
    plot!(ylabel="Average", bottommargin=2Plots.mm)
    savefig("img/yield-paths.pdf")
end

"C:\\Users\\o6m1g\\Documents\\GitHub\\SMC-LGO-CV-private\\img\\yield-paths.pdf"

### Runtime

In [497]:
let
    particles = load("output/DNS/particles.jld")["data"]
    L = size(particles, 2)-1
    ϕ_history = NamedArray(
        load("output/DNS/paths.jld")["data"],
        (leave_ends, 0:L),
        (:T, :l))
    
    times_smc = NamedArray(load("output/DNS/times.jld")["data"], leave_ends, :T)
    times_mcmc = load("output/DNS/times-LEO.jld")["data"]

    _data = cumsum([times_mcmc times_smc times_psis]; dims=1)
    # _data = [_data times_mcmc]
    plot(leave_ends, _data/60,
        label=["MCMC-LEO" "SMC-LEO" "PSIS-LEO"],
        color=[:blue :red :green],
        linestyle=[:solid :dot :dot],
        mswidth=0, markershape=[:pixel :o :square], markersize=3,
        linewidth=2, alpha=[1 0.8 0.7], legend=:topright,
        xticks=(leave_ends,
            df.index[leave_ends] .|> (d -> month(d) == 1 ?  "$(month(d))\n($(year(d)))" : month(d))),
        ylabel="Runtime [min.]",
    )
    for T in leave_ends
        #annotate!(T, times_psis[:T => T] + 3,
        #    text(k̂s_psis[:T => T] > 0.7 ? L"\hat{k}>0.7" : "", :black, 4))
    end
    plot!(bottommargin=2Plots.mm)
    savefig("img/yield-runtime.pdf")
end

"C:\\Users\\o6m1g\\Documents\\GitHub\\SMC-LGO-CV-private\\img\\yield-runtime.pdf"